In [ ]:
import pandas as pd
from pathlib import Path

# Paths
input_path = Path("elections.csv")
output_path = Path("elections_with_perks.csv")

# Perks domain
perks = [
    'Perks.ATimeForGiving', 'Perks.ArcaneCatalyst', 'Perks.AstralNegotiator', 'Perks.Benediction',
    'Perks.BloomingBusiness', 'Perks.Bribe', 'Perks.ChivalrousCarnival', 'Perks.DarkerAuctions',
    'Perks.DoubleMobsHP', 'Perks.DoubleTrouble', 'Perks.EZPZ', 'Perks.ExtraEvent',
    'Perks.ExtraEventFishing_Festival', 'Perks.ExtraEventMining_Fiesta', 'Perks.ExtraEventSpooky_Festival',
    'Perks.ExtraEventSweet_Tooth', 'Perks.FishingFestival', 'Perks.FishingXPBuff', 'Perks.GOATed',
    'Perks.Jerrypocalypse', 'Perks.LongTermInvestment', 'Perks.LuckOfTheSea', 'Perks.Lucky',
    'Perks.MagicXPBoost', 'Perks.Marauder', 'Perks.MiningFiesta', 'Perks.MiningXPBuff', 'Perks.MoarSkillz',
    'Perks.MoltenForge', 'Perks.MythologicalRitual', 'Perks.Pathfinder', 'Perks.PeltPocalypse',
    'Perks.Perkpocalypse', 'Perks.PestEradicator', 'Perks.PetXPBuff', 'Perks.Prospection', 'Perks.QuadTaxes',
    'Perks.SharingIsCaring', 'Perks.ShoppingSpree', 'Perks.SlashedPricing', 'Perks.SlayerXPBuff',
    'Perks.Statspocalypse', 'Perks.StockExchange', 'Perks.SweetBenevolence', 'Perks.TurboMinions',
    'Perks.VolumeTrading'
]

# Load the CSV
if not input_path.exists():
    raise FileNotFoundError(f"Input file not found: {input_path.resolve()}")

df = pd.read_csv(input_path)

# Add missing perk columns with 0
for perk in perks:
    if perk not in df.columns:
        df[perk] = 0

# Save updated CSV
df.to_csv(output_path, index=False)
print(f"Wrote {output_path} with {len(perks)} perk columns added if missing.")


Wrote elections_with_perks.csv with 46 perk columns added if missing.


In [5]:
import pandas as pd
from pathlib import Path

# Files
perks_csv = Path("elections_with_perks.csv")
if not perks_csv.exists():
    raise FileNotFoundError(f"Missing {perks_csv.resolve()} — run the earlier step to create it.")

# Mayor → Perks mapping (provided)
mayor_to_perks = {
  "Aatrox": ["Perks.Pathfinder", "Perks.SlashedPricing", "Perks.SlayerXPBuff"],
  "Barry": ["Perks.ArcaneCatalyst", "Perks.AstralNegotiator", "Perks.MagicXPBoost"],
  "Cole": ["Perks.MiningFiesta", "Perks.MiningXPBuff", "Perks.MoltenForge", "Perks.Prospection"],
  "Derpy": ["Perks.DoubleMobsHP", "Perks.MoarSkillz", "Perks.QuadTaxes", "Perks.TurboMinions"],
  "Diana": ["Perks.Lucky", "Perks.MythologicalRitual", "Perks.PetXPBuff", "Perks.SharingIsCaring"],
  "Diaz": ["Perks.LongTermInvestment", "Perks.ShoppingSpree", "Perks.StockExchange", "Perks.VolumeTrading"],
  "Finnegan": ["Perks.BloomingBusiness", "Perks.GOATed", "Perks.PeltPocalypse", "Perks.PestEradicator"],
  "Foxy": ["Perks.ATimeForGiving", "Perks.ChivalrousCarnival", "Perks.ExtraEventFishing_Festival", "Perks.ExtraEventMining_Fiesta", "Perks.ExtraEventSpooky_Festival", "Perks.ExtraEventSweet_Tooth", "Perks.SweetBenevolence"],
  "Jerry": ["Perks.Jerrypocalypse", "Perks.Perkpocalypse", "Perks.Statspocalypse"],
  "Marina": ["Perks.DoubleTrouble", "Perks.FishingFestival", "Perks.FishingXPBuff", "Perks.LuckOfTheSea"],
  "Paul": ["Perks.Benediction", "Perks.EZPZ", "Perks.Marauder"],
  "Scorpius": ["Perks.Bribe", "Perks.DarkerAuctions"]
}

# Build lookups
# 1) perk -> mayor group label
perk_to_group = {}
for mayor, perks in mayor_to_perks.items():
    for p in perks:
        perk_to_group[p] = mayor

# Load CSV with existing perk columns
df = pd.read_csv(perks_csv)

# Identify all perk columns present (those in the mapping and in df)
perk_columns = [p for p in perk_to_group.keys() if p in df.columns]

# Function to apply weights for a single row
def apply_weights(row):
    # Reset all mapped perk columns to 0 for this row
    for p in perk_columns:
        row[p] = 0.0

    raw_perks = str(row.get("perks", ""))
    if not raw_perks:
        return row

    # Parse perks present in this row and filter to mapped perks
    row_perks = [x.strip() for x in raw_perks.split(",") if x.strip() in perk_to_group]

    # Count how many perks from each mayor's group are present in THIS row
    group_counts = {}
    for p in row_perks:
        g = perk_to_group[p]
        group_counts[g] = group_counts.get(g, 0) + 1

    # Assign weights: 1 / (count of perks in that group in this row)
    for p in row_perks:
        g = perk_to_group[p]
        denom = group_counts.get(g, 0)
        if denom > 0:
            row[p] = 1.0 / denom

    return row

# Apply to every row
df = df.apply(apply_weights, axis=1)

# Save back
df.to_csv(perks_csv, index=False)
print(f"Updated weighted perks written to {perks_csv}")


Updated weighted perks written to elections_with_perks.csv


In [ ]:
import re
import pandas as pd
from pathlib import Path

# Inputs
lua_path = Path("mayor_data.txt")
perks_csv = Path("elections_with_perks.csv")
if not lua_path.exists():
    raise FileNotFoundError(f"Missing {lua_path.resolve()}")
if not perks_csv.exists():
    raise FileNotFoundError(f"Missing {perks_csv.resolve()}")

# Mayor → Perks mapping (same as earlier cell)
mayor_to_perks = {
  "Aatrox": ["Perks.Pathfinder", "Perks.SlashedPricing", "Perks.SlayerXPBuff"],
  "Barry": ["Perks.ArcaneCatalyst", "Perks.AstralNegotiator", "Perks.MagicXPBoost"],
  "Cole": ["Perks.MiningFiesta", "Perks.MiningXPBuff", "Perks.MoltenForge", "Perks.Prospection"],
  "Derpy": ["Perks.DoubleMobsHP", "Perks.MoarSkillz", "Perks.QuadTaxes", "Perks.TurboMinions"],
  "Diana": ["Perks.Lucky", "Perks.MythologicalRitual", "Perks.PetXPBuff", "Perks.SharingIsCaring"],
  "Diaz": ["Perks.LongTermInvestment", "Perks.ShoppingSpree", "Perks.StockExchange", "Perks.VolumeTrading"],
  "Finnegan": ["Perks.BloomingBusiness", "Perks.GOATed", "Perks.PeltPocalypse", "Perks.PestEradicator"],
  "Foxy": ["Perks.ATimeForGiving", "Perks.ChivalrousCarnival", "Perks.ExtraEventFishing_Festival", "Perks.ExtraEventMining_Fiesta", "Perks.ExtraEventSpooky_Festival", "Perks.SweetBenevolence"],
  "Jerry": ["Perks.Jerrypocalypse", "Perks.Perkpocalypse", "Perks.Statspocalypse"],
  "Marina": ["Perks.DoubleTrouble", "Perks.FishingFestival", "Perks.FishingXPBuff", "Perks.LuckOfTheSea"],
  "Paul": ["Perks.Benediction", "Perks.EZPZ", "Perks.Marauder"],
  "Scorpius": ["Perks.Bribe", "Perks.DarkerAuctions"]
}

# Build helpers
perk_to_group = {}
group_to_perks = {}
for mayor, perks in mayor_to_perks.items():
    group_to_perks[mayor] = set(perks)
    for p in perks:
        perk_to_group[p] = mayor

# Parse Lua file to extract minister perks per election
election_to_minister_perks = {}
current_election = None
in_minister = False
brace_depth = 0
perk_regex = re.compile(r"Perks\.[A-Za-z_]+")
num_regex = re.compile(r"^\s*\[(\d+)\]\s*=")

with lua_path.open("r", encoding="utf-8") as f:
    for line in f:
        # detect election start
        m = num_regex.search(line)
        if m:
            current_election = int(m.group(1))
            in_minister = False
            brace_depth = 0
            continue

        if current_election is None:
            continue

        # enter minister block
        if not in_minister and "minister" in line and "{" in line:
            in_minister = True
            brace_depth = line.count("{") - line.count("}")
            # capture any perks on same line
            for perk in perk_regex.findall(line):
                election_to_minister_perks.setdefault(current_election, set()).add(perk)
            continue

        if in_minister:
            # update depth and collect perks
            brace_depth += line.count("{")
            for perk in perk_regex.findall(line):
                election_to_minister_perks.setdefault(current_election, set()).add(perk)
            brace_depth -= line.count("}")
            if brace_depth <= 0:
                in_minister = False
                # finished minister block

# Load CSV and ensure needed columns exist
df = pd.read_csv(perks_csv)
all_mapped_perks = set(perk_to_group.keys())
for p in all_mapped_perks:
    if p not in df.columns:
        df[p] = 0.0

# Apply minister perk overrides per row
def apply_minister_overrides(row):
    try:
        election_num = int(row.get("election_number"))
    except (TypeError, ValueError):
        return row
    minister_perks = election_to_minister_perks.get(election_num)
    if not minister_perks:
        return row

    for perk in minister_perks:
        if perk not in perk_to_group:
            continue
        group = perk_to_group[perk]
        # set target perk to 1
        if perk in row.index:
            row[perk] = 1.0
        # set all other perks in the same group to 0
        for other in group_to_perks[group]:
            if other == perk:
                continue
            if other in row.index:
                row[other] = 0.0
    return row

# Update rows
df = df.apply(apply_minister_overrides, axis=1)

df.to_csv(perks_csv, index=False)
print("Applied minister overrides to", perks_csv)
